# Recherche de production solaire

In [1]:
import pandas as pd
import requests
import numpy as np
import datetime as dt

df = pd.read_csv("data/prod_solaire.csv")

df

FileNotFoundError: [Errno 2] No such file or directory: 'data/prod_solaire.csv'

In [ ]:
df.loc[(df.iloc[:, 1] < 100)  & (df.iloc[:, 1] > 0)]
types = ["hydro", "eolienne", "solar"]
print(f"Please enter any of types in this list : {types} ")

Please enter any of types in this list : ['hydro', 'eolienne', 'solar'] 


In [ ]:
df.describe()

,prod_solaire
count,3310.000000
mean,47.797643
std,30.228459
min,0.000000
25%,23.341965
50%,43.828634
75%,71.486126
max,551.908083


In [ ]:
df_missing = df.isnull().sum()

display(df_missing)

df_ext = df.sort_values(by=["prod_solaire"], ascending=True)


date            0
prod_solaire    2
dtype: int64

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.loc[df["prod_solaire"] > 0].sort_values(by="prod_solaire")

df["date"].dt.dayofweek.value_counts()



date
3    474
4    474
5    474
0    473
2    473
6    472
1    472
Name: count, dtype: int64

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"start_date": "2016-09-01",
	"end_date": "2025-09-25",
	"hourly": ["global_tilted_irradiance", "temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "wind_direction_10m"],
	"tilt": 35,
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_global_tilted_irradiance = hourly.Variables(0).ValuesAsNumpy()
hourly_temperature_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_global_tilted_irradiance)
gti_df = hourly_dataframe["global_tilted_irradiance"]

print(gti_df.info())
print(gti_df.describe())

Coordinates: 43.620384216308594°N 3.930722951889038°E
Elevation: 58.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
 [0. 0. 0. ... 0. 0. 0.]
<class 'pandas.core.series.Series'>
RangeIndex: 79488 entries, 0 to 79487
Series name: global_tilted_irradiance
Non-Null Count  Dtype  
--------------  -----  
79488 non-null  float32
dtypes: float32(1)
memory usage: 310.6 KB
None
count    79488.000000
mean       207.878891
std        296.646332
min          0.000000
25%          0.000000
50%          8.355879
75%        383.465149
max       1074.020630
Name: global_tilted_irradiance, dtype: float64


In [ ]:
TYPES = ("hydro", "eolienne", "solaire")
import pandas as pd

df = pd.read_csv("data/raw/prod_hydro.csv")

def clean(df: pd.DataFrame, energy_type: str) -> pd.DataFrame:
      df.iloc[:, 1] = df.iloc[:, 1].abs()
      if energy_type not in TYPES:
        print(f"Please enter any of types in this list : {TYPES} ")
      elif energy_type == "hydro":
        df = df.loc[(df.iloc[:, 1] <= 200)  & (df.iloc[:, 1] > 0)].copy()
        df = df.rename(columns={"date_obs_elab" : "date"})
      elif energy_type == "eolienne":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
      elif energy_type == "solaire":
        df = df.loc[(df.iloc[:, 1] <= 100)  & (df.iloc[:, 1] > 0)].copy()
        df.iloc[:, 1] = df.iloc[:, 1]*1.5
      df = df.drop_duplicates(subset="date", keep="last")
      df = df.sort_values(by="date")
      df["date"] = pd.to_datetime(df["date"])
      df = df.dropna().reset_index(drop=True)
      df["date"] = df["date"].dt.strftime("%Y-%m-%dT%H:%M:%SZ")
      return df

df_clean = clean(df, "hydro")
display(df_clean)
records = df.to_dict(orient="records")
print(records)

# df_clean.loc[df_clean["prod_hydro"].duplicated() == True]
# df_clean


,date,prod_hydro
0,2022-07-07T00:00:00Z,100.823866
1,2022-07-08T00:00:00Z,111.467475
2,2022-07-09T00:00:00Z,196.434169
3,2022-07-10T00:00:00Z,200.000000
4,2022-07-11T00:00:00Z,200.000000
...,...,...
915,2025-04-01T00:00:00Z,200.000000
916,2025-05-01T00:00:00Z,200.000000
917,2025-06-01T00:00:00Z,200.000000
918,2025-07-01T00:00:00Z,200.000000


[{'date_obs_elab': '2022-08-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2022-09-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2022-10-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2022-11-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2022-12-01', 'prod_hydro': 0.0}, {'date_obs_elab': '2023-01-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-02-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-03-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-04-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-06-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-07-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-08-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-09-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-10-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-11-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2023-12-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2024-01-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2024-02-01', 'prod_hydro': 200.0}, {'date_obs_elab': '2024-03-01

In [ ]:
import os

if os.access(os.getcwd()+"/data/train", os.F_OK) is False:
    os.makedirs(str(os.getcwd()) + "/data/train")

type(os.getcwd())

str

In [ ]:
from supabase import create_client
import os
import pandas as pd
from dotenv import load_dotenv
import json
from sqlalchemy import create_engine, MetaData, Table, Column, Float, Integer, DateTime, text
from sqlalchemy.pool import NullPool

load_dotenv()
types = ("hydro", "solaire", "eolienne")
TYPES = os.getenv("types")
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")
URL = os.getenv("SUPABASE_URL")
SERVICE_ROLE_KEY = os.getenv("SUPABASE_SERVICE_ROLE_KEY")
DATABASE_URL = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}?sslmode=require"

client = create_client(URL, SERVICE_ROLE_KEY)
engine = create_engine(DATABASE_URL, poolclass=NullPool)
meta = MetaData()

def fetch_data(energy_type:str, file_path:str = os.path.join(os.getcwd(), "data/train/")):
        if os.access(file_path, os.F_OK) is False:
            os.makedirs(file_path)
        if energy_type in (None, "solaire"):
            with engine.begin() as conn:
                solaire_table = Table("Solaire_data", meta, autoload_with=engine)
                result_solaire = conn.execute(solaire_table.select())
                df_solaire = pd.DataFrame(sorted(result_solaire))
                return df_solaire.to_csv(file_path+"solaire_train.csv")


                

            # data_solaire = (client.table("Solaire_data").select("*").order("date", desc=False).execute())
            # print(data_solaire.data[1])
            # for row in range(len(data_solaire.data)):
            #     df_solaire = pd.DataFrame(data_solaire.data, columns = data_solaire.data[row].keys())
            #     df_solaire = df_solaire.sort_values(by="id").reset_index(drop=True)
            # return df_solaire
            # df_solaire.to_csv(file_path+"solaire_train.csv")
        if energy_type in (None, "eolienne"):
            data_eolienne = (client.table("Eolienne_data").select("*").execute())
            df_eolienne = pd.DataFrame(data_eolienne)
            df_eolienne.to_csv(file_path+"eolienne_train.csv")
        if energy_type in (None, "hydro"):
            data_hydro = (client.table("Hydro_data").select("*").execute())
            df_hydro = pd.DataFrame(data_hydro)
            df_hydro.to_csv(file_path+"hydro_train.csv")

fetch_data("solaire")

(1, datetime.datetime(2016, 9, 1, 0, 0), 116.89480873521, None, None)
(2, datetime.datetime(2016, 9, 2, 0, 0), 108.257255308276, None, None)
(3, datetime.datetime(2016, 9, 3, 0, 0), 48.479029623941, None, None)
(4, datetime.datetime(2016, 9, 4, 0, 0), 34.7694729679042, None, None)
(5, datetime.datetime(2016, 9, 5, 0, 0), 101.679381426014, None, None)
(6, datetime.datetime(2016, 9, 6, 0, 0), 102.827516532585, None, None)
(7, datetime.datetime(2016, 9, 7, 0, 0), 131.470928402069, None, None)
(8, datetime.datetime(2016, 9, 8, 0, 0), 106.569529848453, None, None)
(9, datetime.datetime(2016, 9, 10, 0, 0), 106.045542355616, None, None)
(10, datetime.datetime(2016, 9, 11, 0, 0), 84.6571639251429, None, None)
(11, datetime.datetime(2016, 9, 12, 0, 0), 88.9812367072502, None, None)
(12, datetime.datetime(2016, 9, 13, 0, 0), 98.9969774626115, None, None)
(13, datetime.datetime(2016, 9, 14, 0, 0), 18.4749420677268, None, None)
(14, datetime.datetime(2016, 9, 15, 0, 0), 65.1178096274799, None, Non

In [ ]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 43.6109,
	"longitude": 3.8763,
	"start_date": "2016-09-01",
	"end_date": "2025-09-25",
	"hourly": ["global_tilted_irradiance", "temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "wind_direction_10m"],
	"tilt": 35,
}
responses = openmeteo.weather_api(url, params=params)

def load_api(energy_type, url, parameters) -> pd.DataFrame:
        
    if energy_type == "solaire":
        all_data = []
        for param in parameters:
            params = {"grandeur_solaire_elab": param, "size": 1500}
            response = requests.get(url, params=params)
            df = pd.DataFrame(response.json().get("data", []))
            if not df.empty:
                df["grandeur_solaire_elab"] = param
                all_data.append(df)
        if all_data:
            return pd.concat(all_data, ignore_index=True)
        return pd.DataFrame()

load_api("solaire", )